In [1]:
from secret_key import openai_key
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [2]:
os.environ['OPENAI_API_KEY'] = openai_key

In [3]:
#Initialise with LLM with required parameters
llm = OpenAI(temperature=0.9, max_tokens=500)

loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html",
    "https://www.moneycontrol.com/news/business/stocks/buy-tata-motors-target-of-rs-743-kr-choksey-11080811.html"
])
data = loaders.load()
len(data)

C:\Users\Tuf\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


2

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)    

#to get chunks we can use document method, as text_splitter gives document type data
docs = text_splitter.split_documents(data)
len(docs)

33

In [5]:
docs[2]

Document(page_content='LearnTECHNOLOGYPersonal TechAutoFintechMEDIAPodcastPhotosInfographicsVideosWeb StoriesCRYPTOCURRENCYOTHERSEntertainmentSportsLifestyleHealth and FitnessEducationJobsScienceAstroTravelT20 WC 2024TechPortfolioWatchlistCommoditiesMutual FundsEXPLOREHomeFind FundTop Ranked FundsPerformance TrackerSIP Performance TrackerETF PerformanceNFOTop Performing CategoriesLearnTOOLSReturns CalculatorLumpsum SIP BalancerDelay Cost CalculatorSIP ReturnMF FORUMTRACKYour MF InvestmentMF PricesMC 30Personal FinanceEXPLOREHomeInvestingInsuranceBankingFinancial PlanningPropertyToolsVideoAsk ExpertExplainerIncome Tax Filing GuideNPSFIXED DEPOSITFixed Deposit ComparisonFixed Deposit Interest CalculatorCorporate DepositsLOANS & CREDIT CARDSHomeLoans\t\t\t\t\t\t\t\t\t\t\t\t\xa0Car Loan CalculatorHome Loan CalculatorEducation Loan CalculatorCredit Card Debit Payoff CalculatorTAXIncome tax Filing GuideIncome Tax CalculatorEmergency Fund CalculatorTOOLSProvident Fund CalculatorAssets Allocat

In [6]:
!pip install tiktoken

In [7]:
embeddings = OpenAIEmbeddings()

vectorindex_openai = FAISS.from_documents(docs, embeddings)

C:\Users\Tuf\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
# Storing vector index create in local
file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex_openai, f)

TypeError: cannot pickle '_thread.RLock' object

In [10]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

EOFError: Ran out of input

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())

In [ ]:
query = "what is the price of Tiago iCNG"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)

Here we can see that we are getting most similar chunks to our query and then getting answer from each chunk, finally outputs the best matching answer.